<a href="https://colab.research.google.com/github/AliyaJabbar/Mazecode/blob/main/hexa_Maze8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import random
import math

# Even-q vertical layout directions (dx, dy) for hex grid
DIRECTIONS = [
    (+1, 0), (+1, -1), (0, -1),
    (-1, 0), (-1, +1), (0, +1)
]

# Maze generation with recursive backtracking
def generate_hex_maze(rows, cols):
    maze = [[0b111111 for _ in range(cols)] for _ in range(rows)]
    visited = [[False for _ in range(cols)] for _ in range(rows)]

    def neighbors(r, c):
        result = []
        for i, (dr, dc) in enumerate(DIRECTIONS):
            nr = r + dr
            nc = c + dc if r % 2 == 0 or dr == 0 else c + dc + 1
            if 0 <= nr < rows and 0 <= nc < cols and not visited[nr][nc]:
                result.append((i, nr, nc))
        return result

    def carve(r, c):
        visited[r][c] = True
        nbrs = neighbors(r, c)
        random.shuffle(nbrs)
        for i, nr, nc in nbrs:
            if not visited[nr][nc]:
                # Remove wall between (r, c) and (nr, nc)
                maze[r][c] &= ~(1 << i)
                maze[nr][nc] &= ~(1 << ((i + 3) % 6))
                carve(nr, nc)

    carve(0, 0)
    return maze

# Pathfinding using DFS
def solve_maze(maze):
    rows, cols = len(maze), len(maze[0])
    stack = [(0, 0)]
    visited = [[False]*cols for _ in range(rows)]
    parent = {}

    while stack:
        r, c = stack.pop()
        if visited[r][c]:
            continue
        visited[r][c] = True
        if (r, c) == (rows-1, cols-1):
            break
        for i, (dr, dc) in enumerate(DIRECTIONS):
            if not (maze[r][c] & (1 << i)):
                nr = r + dr
                nc = c + dc if r % 2 == 0 or dr == 0 else c + dc + 1
                if 0 <= nr < rows and 0 <= nc < cols and not visited[nr][nc]:
                    stack.append((nr, nc))
                    parent[(nr, nc)] = (r, c)

    # Build path
    path = []
    curr = (rows - 1, cols - 1)
    while curr != (0, 0):
        path.append(curr)
        curr = parent[curr]
    path.append((0, 0))
    return path[::-1]

# Convert hex grid to Cartesian coordinates
def hex_to_pixel(r, c, size):
    x = size * math.sqrt(3) * (c + 0.5 * (r % 2))
    y = size * 1.5 * r
    return x, y

# Draw the maze
def draw_maze(maze, path=None, filename='hex_maze.png'):
    rows, cols = len(maze), len(maze[0])
    size = 10
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.set_aspect('equal')
    ax.axis('off')

    for r in range(rows):
        for c in range(cols):
            cx, cy = hex_to_pixel(r, c, size)
            angles = [math.radians(60 * i) for i in range(6)]
            points = [(cx + size * math.cos(a), cy + size * math.sin(a)) for a in angles]
            for i in range(6):
                if maze[r][c] & (1 << i):
                    x0, y0 = points[i]
                    x1, y1 = points[(i + 1) % 6]
                    ax.plot([x0, x1], [y0, y1], color='black', lw=1)

    if path:
        px = []
        py = []
        for r, c in path:
            x, y = hex_to_pixel(r, c, size)
            px.append(x)
            py.append(y)
        ax.plot(px, py, color='blue', lw=2)
        ax.scatter(px[0], py[0], color='green', s=100, label='Start', zorder=5)
        ax.scatter(px[-1], py[-1], color='red', s=100, label='End', zorder=5)

    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    Image.open(filename).save(filename.replace('.png', '.pdf'))
    plt.close()

# ===== RUN =====
ROWS, COLS = 10, 10
maze = generate_hex_maze(ROWS, COLS)
path = solve_maze(maze)

draw_maze(maze, path=None, filename='hex_maze_question.png')  # Question only
draw_maze(maze, path=path, filename='hex_maze_solution.png')  # Solution with path


